# Refactor the Taxi Fare Prediction Problem with a Pipeline

We will refactor the model you built for the Taxi Fare Prediction Problem using:
- Custom encoders for the distance and time features
- OneHot Encoder in order to encode the hour and day of week features
- SimpleImputer to fill missing values
- A simple linear regression
- A pipeline to put all together

Then: 
- train this pipeline
- apply the pipeline on test data
- generate predictions and submit these new predictions to Kaggle

In [1]:
import pandas as pd

## First pipeline

In [3]:
# import the train dataset ( 1000 first rows)
df = pd.read_csv('data/train.csv', nrows= 1_000)
df

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1
...,...,...,...,...,...,...,...,...
995,2011-07-30 07:33:13.0000001,16.9,2011-07-30 07:33:13 UTC,-73.884801,40.755707,-73.980472,40.765556,1
996,2011-09-24 23:20:00.000000112,13.7,2011-09-24 23:20:00 UTC,-73.953603,40.779203,-73.995763,40.726701,3
997,2011-12-22 11:07:00.00000037,29.3,2011-12-22 11:07:00 UTC,-73.942380,40.837712,-73.864372,40.769985,2
998,2014-11-03 12:40:00.00000014,6.5,2014-11-03 12:40:00 UTC,-73.961151,40.774578,-73.972251,40.785640,1


In [4]:
# prepare X and y
X = df.loc[ : , df.columns != 'fare_amount']
y = df.fare_amount

In [5]:
X

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1
...,...,...,...,...,...,...,...
995,2011-07-30 07:33:13.0000001,2011-07-30 07:33:13 UTC,-73.884801,40.755707,-73.980472,40.765556,1
996,2011-09-24 23:20:00.000000112,2011-09-24 23:20:00 UTC,-73.953603,40.779203,-73.995763,40.726701,3
997,2011-12-22 11:07:00.00000037,2011-12-22 11:07:00 UTC,-73.942380,40.837712,-73.864372,40.769985,2
998,2014-11-03 12:40:00.00000014,2014-11-03 12:40:00 UTC,-73.961151,40.774578,-73.972251,40.785640,1


In [6]:
y

0       4.5
1      16.9
2       5.7
3       7.7
4       5.3
       ... 
995    16.9
996    13.7
997    29.3
998     6.5
999    10.9
Name: fare_amount, Length: 1000, dtype: float64

In [7]:
from sklearn.model_selection import train_test_split
# Hold out ( train and test dplit )
X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size=0.1)

In [8]:
X_train.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
937,2009-03-19 16:15:36.0000004,2009-03-19 16:15:36 UTC,-74.000548,40.737311,-73.993162,40.734975,1
901,2012-10-05 13:34:41.0000002,2012-10-05 13:34:41 UTC,-73.956865,40.774874,-73.955842,40.784669,1
510,2010-10-23 18:39:00.000000208,2010-10-23 18:39:00 UTC,-73.782215,40.644272,-73.782217,40.644270,2
701,2010-03-22 14:59:45.0000001,2010-03-22 14:59:45 UTC,-73.961353,40.765069,-73.960520,40.769931,1
404,2014-02-06 11:17:00.00000070,2014-02-06 11:17:00 UTC,-73.980710,40.782635,-74.002040,40.719425,1


### Custom transformers

With the Taxi Fare Prediction Challenge data, using `BaseEstimator` and `TransformerMixin`, implement:

- a transformer that computes the haversine distance between the pickup and dropoff locations
- a custom encoder that extracts the time features from `pickup_datetime`

In [9]:
import numpy as np

def haversine_vectorized(df, 
                         start_lat="pickup_latitude",
                         start_lon="pickup_longitude",
                         end_lat="dropoff_latitude",
                         end_lon="dropoff_longitude"):
    """ 
        Calculates the great circle distance between two points 
        on the earth (specified in decimal degrees).
        Vectorized version of the haversine distance for pandas df.
        Computes the distance in kms.
    """

    lat_1_rad, lon_1_rad = np.radians(df[start_lat].astype(float)), np.radians(df[start_lon].astype(float))
    lat_2_rad, lon_2_rad = np.radians(df[end_lat].astype(float)), np.radians(df[end_lon].astype(float))
    dlon = lon_2_rad - lon_1_rad
    dlat = lat_2_rad - lat_1_rad

    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat_1_rad) * np.cos(lat_2_rad) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return 6371 * c

In [10]:
# create a DistanceTransformer
from sklearn.base import BaseEstimator, TransformerMixin

class DistanceTransformer(BaseEstimator, TransformerMixin):
    """
        Computes the haversine distance between two GPS points.
        Returns a copy of the DataFrame X with only one column: 'distance'.
    """

    def __init__(self, start_lat="pickup_latitude", start_lon="pickup_longitude", 
                 end_lat="dropoff_latitude", end_lon="dropoff_longitude"):
        self.start_lat = start_lat
        self.start_lon = start_lon
        self.end_lat = end_lat
        self.end_lon = end_lon
    
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_ = X.copy()
        X_['distance'] = haversine_vectorized(X_, start_lat = self.start_lat, 
                                              start_lon = self.start_lon,
                                              end_lat = self.end_lat,
                                              end_lon = self.end_lon)
    
        return X_[['distance']]

In [11]:
# test the DistanceTransformer

dist_trans = DistanceTransformer()
distance = dist_trans.fit_transform(X_train, y_train)
distance.head()

,distance
937,0.674342
901,1.092555
510,0.000279
701,0.545162
404,7.254653


In [12]:
def extract_time_features(df):
    
    df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], format='%Y-%m-%d %H:%M:%S UTC')
    df['hour'] = df['pickup_datetime'].dt.hour
    df['dow'] = df['pickup_datetime'].dt.dayofweek
    df['month'] = df['pickup_datetime'].dt.month
    df['year'] = df['pickup_datetime'].dt.year
    return df

In [13]:
# create a TimeFeaturesEncoder
class TimeFeaturesEncoder(BaseEstimator, TransformerMixin):
    """
        Extracts the day of week (dow), the hour, the month and the year from a time column.
        Returns a copy of the DataFrame X with only four columns: 'dow', 'hour', 'month', 'year'.
    """

    def __init__(self,time_column='pickup_datetime'):
        self.time_column = time_column

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_ = X.copy()
        X_ = extract_time_features(X)
        return X_[['dow','hour', 'month', 'year']]

In [14]:
# test the TimeFeaturesEncoder

time_enc = TimeFeaturesEncoder('pickup_datetime')
time_features = time_enc.fit_transform(X_train, y_train)
time_features.head()

,dow,hour,month,year
937,3,16,3,2009
901,4,13,10,2012
510,5,18,10,2010
701,0,14,3,2010
404,3,11,2,2014


###  Prepocessing pipeline

In [15]:
# visualizing pipelines in HTML
from sklearn import set_config; set_config(display='diagram')
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
import math

#### Distance pipeline

Create a pipeline for distances:
- convert the pickup and dropoff coordinates into distances with the DistanceTransformer
- standardize these distances

In [16]:
# create distance pipeline dist_pipe
dist_pipe = Pipeline([('dist_trans', DistanceTransformer()),('stdscaler', StandardScaler())])

# display distance pipeline
dist_pipe

Pipeline(steps=[('dist_trans', DistanceTransformer()),
                ('stdscaler', StandardScaler())])

#### Time features pipeline

Create a pipeline for time features
- extract time features from pickup datetime with the TimeFeaturesEncoder
- encode these categorical time features with the OneHotEncoder

In [17]:

# create time pipeline time_pipe
time_pipe = Pipeline([('time_enc', TimeFeaturesEncoder()),('ohe', OneHotEncoder(handle_unknown='ignore'))])

# display time pipeline
time_pipe

Pipeline(steps=[('time_enc', TimeFeaturesEncoder()),
                ('ohe', OneHotEncoder(handle_unknown='ignore'))])

#### Preprocessing pipeline

Wrap up the distance pipeline and the time pipeline into a preprocessing pipeline.

In [18]:
# create preprocessing pipeline preproc_pipe
preproc_pipe = ColumnTransformer(
    transformers=[
        ('distance', Pipeline(steps=[('dist_trans',DistanceTransformer()),
                                     ('stdscaler',StandardScaler())]),
                                 ['pickup_latitude', 'pickup_longitude',
                                  'dropoff_latitude', 'dropoff_longitude']),
        ('time', Pipeline(steps=[('time_enc',TimeFeaturesEncoder()),
                                ('ohe',OneHotEncoder(handle_unknown='ignore'))]),
                                 ['pickup_datetime'])])
# display preprocessing pipeline
preproc_pipe.fit(X_train, y_train)

ColumnTransformer(transformers=[('distance',
                                 Pipeline(steps=[('dist_trans',
                                                  DistanceTransformer()),
                                                 ('stdscaler',
                                                  StandardScaler())]),
                                 ['pickup_latitude', 'pickup_longitude',
                                  'dropoff_latitude', 'dropoff_longitude']),
                                ('time',
                                 Pipeline(steps=[('time_enc',
                                                  TimeFeaturesEncoder()),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['pickup_datetime'])])

In [19]:
# create preprocessing pipeline preproc_pipe
preproc_pipe = ColumnTransformer(
    transformers=[
        ('distance', dist_pipe,['pickup_latitude', 'pickup_longitude',
                                  'dropoff_latitude', 'dropoff_longitude']),
        ('time', time_pipe, ['pickup_datetime'])])
# display preprocessing pipeline
preproc_pipe

ColumnTransformer(transformers=[('distance',
                                 Pipeline(steps=[('dist_trans',
                                                  DistanceTransformer()),
                                                 ('stdscaler',
                                                  StandardScaler())]),
                                 ['pickup_latitude', 'pickup_longitude',
                                  'dropoff_latitude', 'dropoff_longitude']),
                                ('time',
                                 Pipeline(steps=[('time_enc',
                                                  TimeFeaturesEncoder()),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['pickup_datetime'])])

### Model pipeline

Create a pipeline containing the preprocessing and the regression model of your choice.

In [20]:
# create preprocessing pipeline preproc_pipe
pipe = Pipeline([('preproc',preproc_pipe),
                ('linear_model', LinearRegression())])
pipe

Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('distance',
                                                  Pipeline(steps=[('dist_trans',
                                                                   DistanceTransformer()),
                                                                  ('stdscaler',
                                                                   StandardScaler())]),
                                                  ['pickup_latitude',
                                                   'pickup_longitude',
                                                   'dropoff_latitude',
                                                   'dropoff_longitude']),
                                                 ('time',
                                                  Pipeline(steps=[('time_enc',
                                                                   TimeFeaturesEncoder()),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['pickup_datetime'])])),
                ('linear_model', LinearRegression())])

### Training and performance

Train the pipelined model and compute the prediction on the test set:

In [21]:
# train the pipelined model
pipe.fit(X_train, y_train)

# compute y_pred on the test set
y_pred = pipe.predict(X_test)

In [22]:
y_test = y_test.values

Use the RMSE to evaluate the performance of the model:

In [23]:
def compute_rmse(y_pred, y_true):
    diff = []
    for i in range(len(y_pred)) : 
        a = (y_pred[i]-y_true[i])**2
        diff.append(a)
        
    RMSE = sum(diff)
    RMSE = RMSE/len(y_pred)
    RMSE =  math.sqrt(RMSE)
    
    return RMSE

In [24]:
# call compute_rmse
compute_rmse(y_pred, y_test)

9.863573804116186

## Complete the workflow with a pipeline

Here we will implement the whole workflow for our Taxifare kaggle challenge.

For that we will refactor the code in functions for more clarity.

Implement the following functions:
- `get_data()` to fetch the data 
- `clean_data()` to clean the data
- `get_pipeline()` to get the pipeline defined earlier
- `train()` to train our model
- `evaluate()` to evaluate our model on test data

In [31]:
# implement get_data() function
def get_data(nrows=10000):
    df = pd.read_csv('data/train.csv', nrows=nrows)
    return df

In [26]:
# implement clean_data() function
def clean_data(df, test=False):
    '''returns a DataFrame without outliers and missing values'''
    df = df[df.passenger_count.between(1, 10)]
    df = df[df.fare_amount.between(0, 60)]
    df = df[df["pickup_latitude"].between(left = 40, right = 42 )]
    df = df[df["pickup_longitude"].between(left = -74.3, right = -72.9 )]
    df = df[df["dropoff_latitude"].between(left = 40, right = 42 )]
    df = df[df["dropoff_longitude"].between(left = -74.3, right = -72.9 )]
    return df

In [27]:
# implement set_pipeline() function
def set_pipeline():
    '''returns a pipelined model'''
    pipe = Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('distance',
                                                  Pipeline(steps=[('dist_trans',
                                                                   DistanceTransformer()),
                                                                  ('stdscaler',
                                                                   StandardScaler())]),
                                                  ['pickup_latitude',
                                                   'pickup_longitude',
                                                   'dropoff_latitude',
                                                   'dropoff_longitude']),
                                                 ('time',
                                                  Pipeline(steps=[('time_enc',
                                                                   TimeFeaturesEncoder(time_column='pickup_datetime')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['pickup_datetime'])])),
                ('linear_model', LinearRegression())])
    return pipe

In [28]:
# implement train() function
def train(X_train, y_train, pipeline):
    '''returns a trained pipelined model'''
    pipeline = pipeline.fit(X_train, y_train)
    
    return pipeline

In [29]:
# implement evaluate() function
def evaluate(X_test, y_test, pipeline):
    '''returns the value of the RMSE'''
    y_pred = pipeline.predict(X_test)
    y_test = y_test.values
    rmse = compute_rmse(y_pred, y_test)
    print(rmse)
    return rmse

### Test the complete worflow

Use the above functions to test the complete workflow.

In [32]:
# store the data in a DataFrame
df = get_data()

#clean data
df = clean_data(df)

# set X and y
y = df["fare_amount"]
X = df.drop("fare_amount", axis=1)

# hold out
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15)

# build pipeline
pipeline = set_pipeline()

# train the pipeline
train(X_train, y_train, pipeline)

# evaluate the pipeline
rmse = evaluate(X_val, y_val, pipeline)

6.091466488794959


### Congrats!

Now we are ready to convert this complete workflow into a packaged code 🚀